# Mask R-CNN - Cityscapes hyperparameters tuning

## Initialization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import modules

In [ ]:
import os
import sys
import json

ROOT_DIR = os.path.abspath("/content/drive/MyDrive/cityscapes/utils")
sys.path.append(ROOT_DIR)

import utils_cityscapes   # Matterport Mask R-CNN set-up
from utils_cityscapes import CityscapesConfig, CityscapesDataset

from mrcnn import model as modellib, utils

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 2.1MB/s 


In [ ]:
import kerastuner as kt
import tensorflow as tf

## Dataset

In [ ]:
DATASET_DIR = "/content/drive/MyDrive/cityscapes/dataset/Tiny"

# Training dataset
dataset_train = CityscapesDataset()
dataset_train.load_cityscapes(DATASET_DIR, "Train")   # Load dataset
dataset_train.prepare()   # Must call before using the dataset

# Validation dataset
dataset_val = CityscapesDataset()
dataset_val.load_cityscapes(DATASET_DIR, "Val")
dataset_val.prepare()

## Configurations

In [ ]:
class HyperParamConfig(CityscapesConfig):
  """Configuration for training on the Cityscapes dataset.
  Derives from the CityscapeConfig class and overrides some values.
  """

  def __init__(self, learning_rate=0.001, weight_decay=0.0001, learning_momentum=0.9, detection_min_confidence=0.7):
    super().__init__()
    self.LEARNING_RATE = learning_rate
    self.WEIGHT_DECAY = weight_decay
    self.LEARNING_MOMENTUM = learning_momentum
    self.DETECTION_MIN_CONFIDENCE = detection_min_confidence

  STEPS_PER_EPOCH = 154 # default 1000
  VALIDATION_STEPS = 30

## HyperModel



In [ ]:
COCO_WEIGHTS_PATH =  "/content/drive/MyDrive/Mask_RCNN/mask_rcnn_coco.h5"

MODELS_DIR = "/content/drive/MyDrive/cityscapes/models"
MODEL_DIR = os.path.join(MODELS_DIR, "from_coco/hyperparam_tuning")

In [ ]:
def model_builder(hp):
  hp_learning_rate = hp.Float('learning_rate', 1e-5, 1e-4)  
  hp_weight_decay = hp.Float('weight_decay', 1e-4, 1e-2)   
  hp_learning_momentum = hp.Float('learning_momentum', 0.9, 0.99)

  config_train = HyperParamConfig(hp_learning_rate, hp_weight_decay, hp_learning_momentum)

  model = modellib.MaskRCNN(mode="training", config=config_train,
                                  model_dir=MODEL_DIR)
  model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

  layers = r"(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)"
  model.set_trainable(layers)
  model.compile(config_train.LEARNING_RATE, config_train.LEARNING_MOMENTUM, optimizer='SGD')

  return model.keras_model

## Random search

In [ ]:
tuner = kt.RandomSearch(
    model_builder,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    directory=MODEL_DIR,
    project_name='random_search')

Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)
mrcnn_class_logits     (TimeDistributed)
mrcnn_mask             (TimeDistributed)
Using SGD optimizer


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
learning_rate (Float)
{'default': 0.001, 'conditions': [], 'min_value': 0.001, 'max_value': 1e-05, 'step': None, 'sampling': None}
weight_decay (Float)
{'default': 0.01, 'conditions': [], 'min_value': 0.01, 'max_value': 0.0001, 'step': None, 'sampling': None}
learning_momentum (Float)
{'default': 0.9, 'conditions': [], 'min_value': 0.9, 'max_value': 0.99, 'step': None, 'sampling': None}


In [ ]:
data_config = HyperParamConfig()

train_generator = modellib.DataGenerator(dataset_train, data_config, shuffle=True, augmentation=None)
val_generator = modellib.DataGenerator(dataset_val, data_config, shuffle=True)

In [ ]:
epochs=5

tuner.search(
    train_generator,
    epochs=epochs,
    steps_per_epoch=data_config.STEPS_PER_EPOCH,
    callbacks=[],
    validation_data=val_generator,
    validation_steps=data_config.VALIDATION_STEPS,
    max_queue_size=100
)

INFO:tensorflow:Oracle triggered exit


## Hyperband

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=20,
                     factor=3,
                     directory=MODEL_DIR,
                     project_name='hyperband')

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 1e-05, 'step': None, 'sampling': None}
weight_decay (Float)
{'default': 0.01, 'conditions': [], 'min_value': 0.01, 'max_value': 0.0001, 'step': None, 'sampling': None}
learning_momentum (Float)
{'default': 0.9, 'conditions': [], 'min_value': 0.9, 'max_value': 0.99, 'step': None, 'sampling': None}


In [ ]:
data_config = HyperParamConfig()

train_generator = modellib.DataGenerator(dataset_train, data_config, shuffle=True, augmentation=None)
val_generator = modellib.DataGenerator(dataset_val, data_config, shuffle=True)

In [ ]:
epochs=200

tuner.search(
    train_generator,
    epochs=epochs,
    steps_per_epoch=data_config.STEPS_PER_EPOCH,
    callbacks=[],
    validation_data=val_generator,
    validation_steps=data_config.VALIDATION_STEPS,
    max_queue_size=100
)

INFO:tensorflow:Oracle triggered exit


## Trials results

In [ ]:
TRIALS_DIR = os.path.join(MODEL_DIR, "hyperband")

trials = []

for root, dirs, files in os.walk(TRIALS_DIR):
  for f in files:
    if f == "trial.json":
      with open(os.path.join(root, f)) as fj:
        j = json.load(fj)
        trials.append(j)

In [ ]:
for t in trials:
  print("hyperparameters:", t["hyperparameters"]["values"])
  print("score:", t["score"])

hyperparameters: {'learning_rate': 0.001, 'weight_decay': 0.01, 'learning_momentum': 0.99, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
score: 1.5831897755463917
hyperparameters: {'learning_rate': 0.0001, 'weight_decay': 0.0001, 'learning_momentum': 0.99, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
score: 1.1625506500403087
hyperparameters: {'learning_rate': 0.0001, 'weight_decay': 0.01, 'learning_momentum': 0.99, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
score: 1.1081172327200572
hyperparameters: {'learning_rate': 0.0001, 'weight_decay': 0.0001, 'learning_momentum': 0.9, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
score: 1.215052925546964
hyperparameters: {'learning_rate': 0.001, 'weight_decay': 0.0001, 'learning_momentum': 0.9, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
score: 1.16597730120